In [19]:
!pip install rdflib

In [46]:
import rdflib

# Load the RDF file
rdf_file = "/Users/chengluotong/Desktop/mapping.rdf" 
graph = rdflib.Graph()
graph.parse(rdf_file)

<Graph identifier=N3cb21c7b74eb4cd09ab6dbd1856da9f6 (<class 'rdflib.graph.Graph'>)>

In [21]:
sparql_query = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX lrmoo: <http://iflastandards.info/ns/lrm/lrmoo/>
PREFIX cidoc-crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX crmdig: <http://www.cidoc-crm.org/extensions/crmdig/>

SELECT ?Identifier ?Title ?VisualFeature ?IIIF_Image
WHERE {
  ?expression cidoc-crm:P165_incorporates ?visualitem .  
  ?visualitem rdf:type cidoc-crm:E36_Visual_Item .
  ?visualitem rdfs:label ?VisualFeature .
  ?manifestation lrmoo:R4_embodies ?expression .
  ?item lrmoo:R7_exemplifies ?manifestation .
  ?digitization crmdig:L1_digitized ?item .
  ?digitization crmdig:L20_has_created ?Digital_Image .
  ?Digital_Image cidoc-crm:P102_has_title ?title .
  ?title rdf:type cidoc-crm:E35_Title .
  ?title rdfs:label ?Title .
  ?Digital_Image cidoc-crm:P48_has_preferred_identifier ?Identifier .
  ?IIIF_Image cidoc-crm:P138_represents ?Digital_Image 
}
"""

results = graph.query(sparql_query)

for result in results:
    Identifier = result.get("Identifier")
    Title = result.get("Title")
    Visual_Feature = result.get("VisualFeature")
    IIIF_Image = result.get("IIIF_Image")

    print("Identifier:", Identifier)
    print("Title:", Title)
    print("Visual Feature:", Visual_Feature)
    print("IIIF Image:", IIIF_Image)

Identifier: http://cdm21053.contentdm.oclc.org/cdm/ref/collection/Kaarten/id/2791
Title: Wereldkaart 1509
Visual Feature: vier windhoofden
IIIF Image: https://dbc.rug.nl/digital/iiif/Kaarten/2791/full/full/0/iiifimg2791


In [22]:
sparql_query = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX lrmoo: <http://iflastandards.info/ns/lrm/lrmoo/>
PREFIX cidoc-crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX crmdig: <http://www.cidoc-crm.org/extensions/crmdig/>

SELECT ?Identifier ?Title ?Description ?Language ?IIIF_Image 
WHERE {
  ?expression cidoc-crm:P165_incorporates ?linguisticItem .  
  ?linguisticItem rdf:type cidoc-crm:E33_Linguistic_Object .
  ?linguisticItem rdfs:label ?Description .
    OPTIONAL { ?linguisticItem cidoc-crm:P72_has_language ?language .
        ?language rdf:type cidoc-crm:E56_Language .
        ?language rdfs:label ?Language}
  ?manifestation lrmoo:R4_embodies ?expression .
  ?item lrmoo:R7_exemplifies ?manifestation .
  ?digitization crmdig:L1_digitized ?item .
  ?digitization crmdig:L20_has_created ?Digital_Image .
  ?Digital_Image cidoc-crm:P102_has_title ?title .
  ?title rdf:type cidoc-crm:E35_Title .
  ?title rdfs:label ?Title .
  ?Digital_Image cidoc-crm:P48_has_preferred_identifier ?Identifier .
  ?IIIF_Image cidoc-crm:P138_represents ?Digital_Image .
   
      }

"""

results = graph.query(sparql_query)

for result in results:
    Identifier = result.get("Identifier")
    Title = result.get("Title")
    Description = result.get("Description")
    Language = result.get("Language")
    IIIF_Image = result.get("IIIF_Image")

    print("Identifier:", Identifier)
    print("Title:", Title)
    print("Description:", Description)
    print("Language:", Language)
    print("IIIF Image:", IIIF_Image)
    print("\n")

Identifier: http://cdm21053.contentdm.oclc.org/cdm/ref/collection/Kaarten/id/1492
Title: Tweede stuk der beleegering van Bergen op Zoom : met de afbeeldinge der Franse attaque op de Linie en Roversfort
Description: Legenda
Language: None
IIIF Image: https://dbc.rug.nl/digital/iiif/Kaarten/1489/full/full/0/iiifimg1492


Identifier: http://cdm21053.contentdm.oclc.org/cdm/ref/collection/Kaarten/id/1224
Title: Afbeeldinge der stadt Groningen met de omliggende fortressen : 1652
Description: Met lijsten van openbare gebouwen en gasthuizen en van straten en poorten
Language: None
IIIF Image: https://dbc.rug.nl/digital/iiif/Kaarten/1223/full/full/0/default.jpg#iiifimg1224


Identifier: http://cdm21053.contentdm.oclc.org/cdm/ref/collection/Kaarten/id/1492
Title: Tweede stuk der beleegering van Bergen op Zoom : met de afbeeldinge der Franse attaque op de Linie en Roversfort
Description: Title in map
Language: Italian
IIIF Image: https://dbc.rug.nl/digital/iiif/Kaarten/1489/full/full/0/iiifimg149

In [45]:
sparql_query = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX lrmoo: <http://iflastandards.info/ns/lrm/lrmoo/>
PREFIX cidoc-crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX crmdig: <http://www.cidoc-crm.org/extensions/crmdig/>

SELECT ?Title ?Identifier ?Cartographer ?Creation_Date ?Creation_Place ?Publisher ?Publication_Date ?Publication_Place ?Source_Title ?Author ?Reference ?ref_uri
WHERE {
  ?map_publication rdf:type lrmoo:F30_Manifestation_Creation .
  ?map_publication lrmoo:R24_created ?edition .
  ?map_creation lrmoo:R17_created ?expression .
  ?edition lrmoo:R4_embodies ?expression .
  ?expression rdf:type lrmoo:F2_Expression .
  ?item lrmoo:R7_exemplifies ?edition .
  ?digitization crmdig:L1_digitized ?item .
  ?digitization crmdig:L20_has_created ?Digital_Image .
  ?Digital_Image cidoc-crm:P102_has_title ?title .
  ?title rdf:type cidoc-crm:E35_Title .
  ?title rdfs:label ?Title .
  ?Digital_Image cidoc-crm:P48_has_preferred_identifier ?Identifier .
OPTIONAL {
    ?map_publication cidoc-crm:P4_has_time-span ?pub_timespan .
    ?pub_timespan rdf:type cidoc-crm:E52_Time-Span .
    ?pub_timespan cidoc-crm:P82_at_some_time_within ?Publication_Date .
    }
  OPTIONAL {
    ?map_creation cidoc-crm:P4_has_time-span ?cre_timespan .
    ?cre_timespan rdf:type cidoc-crm:E52_Time-Span .
    ?cre_timespan cidoc-crm:P82_at_some_time_within ?Creation_Date .
    }
  OPTIONAL{
      ?map_publication cidoc-crm:P14_carried_out_by ?publisher .
      ?publisher rdf:type cidoc-crm:E39_Actor .
      ?publisher rdfs:label ?Publisher
    }
  OPTIONAL{
      ?map_creation cidoc-crm:P14_carried_out_by ?cartographer .
      ?cartographer rdf:type cidoc-crm:E39_Actor .
      ?cartographer rdfs:label ?Cartographer
    } 
  OPTIONAL{
      ?map_publication cidoc-crm:P7_took_place_at ?publication_place .
      ?publication_place rdf:type cidoc-crm:E53_Place .
      ?publication_place rdfs:label ?Publication_Place
    }
  OPTIONAL{
      ?map_creation cidoc-crm:P7_took_place_at ?creation_place .
      ?creation_place rdf:type cidoc-crm:E53_Place .
      ?creation_place rdfs:label ?Creation_Place
        }
  OPTIONAL {
       ?original_source lrmoo:R71_has_part ?edition .
       ?original_source cidoc-crm:P102_has_title ?sourceTitle .
       ?sourceTitle rdfs:label ?Source_Title .
       ?original_source lrmoo:R4_embodies ?source_expression .
       ?source_creation lrmoo:R17_created ?source_expression .
       ?source_creation cidoc-crm:P14_carried_out_by ?author .
       ?author rdfs:label ?Author .
    }
  OPTIONAL {
       ?referencePage cidoc-crm:P130_shows_features_of ?Digital_Image .
       ?referencePage rdfs:label ?Reference .
       ?reference cidoc-crm:P106_is_composed_of ?referencePage .
       ?reference rdf:type lrmoo:F5_Item .
       ?reference cidoc-crm:P48_has_preferred_identifier ?ref_uri .       
    }
  
}
"""

results = graph.query(sparql_query)
for result in results:
    title = result.get("Title")
    identifier = result.get("Identifier")
    cartographer = result.get("Cartographer")
    creation_date = result.get("Creation_Date")
    creation_place = result.get("Creation_Place")
    publisher = result.get("Publisher")
    publication_date = result.get("Publication_Date")
    publication_place = result.get("Publication_Place")
    Source_Title = result.get("Source_Title")
    Author = result.get("Author")
    Reference = result.get("Reference")

    
    print("Title:", title)
    print("Identifier:", identifier)
    print("Cartographer:", cartographer)
    print("Creation Date:", creation_date)
    print("Creation Place:", creation_place)
    print("Publisher:", publisher)
    print("Publication Date:", publication_date)
    print("Publication Place:", publication_place)
    print("Source_Title:", Source_Title)
    print("Author:", Author)
    print ("Reference:", Reference)
    print("\n")

Title: Afbeeldinge der stadt Groningen met de omliggende fortressen : 1652
Identifier: http://cdm21053.contentdm.oclc.org/cdm/ref/collection/Kaarten/id/1224
Cartographer: Egbert Haubois
Creation Date: 1652
Creation Place: None
Publisher: None
Publication Date: 1694
Publication Place: None
Source_Title: None
Author: None
Reference: None


Title: Tweede stuk der beleegering van Bergen op Zoom : met de afbeeldinge der Franse attaque op de Linie en Roversfort
Identifier: http://cdm21053.contentdm.oclc.org/cdm/ref/collection/Kaarten/id/1492
Cartographer: Daniel Langeweg
Creation Date: None
Creation Place: None
Publisher: Daniel Langeweg
Publication Date: 1747
Publication Place: Den Haag
Source_Title: None
Author: None
Reference: Van Rijn, Atlas van Stolk, dl. 5 p. 83, 3707


Title: Grand theatre de la guerre sur les frontieres de France ÿ& d'Allemagne ou carte nouvelle du cours du Rhin et des païs circonvoisins contenant la Haute ÿ Basse Alsace, ...
Identifier: http://cdm21053.contentdm.ocl

In [24]:
sparql_query = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX lrmoo: <http://iflastandards.info/ns/lrm/lrmoo/>
PREFIX cidoc-crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX crmdig: <http://www.cidoc-crm.org/extensions/crmdig/>

SELECT ?actor ?Actor ?Role ?Title ?Date
WHERE {
  ?actor rdf:type cidoc-crm:E39_Actor .
  ?actor rdfs:label ?Actor .
  ?actor cidoc-crm:P2_has_type ?role .
  ?role rdfs:label ?Role .
  OPTIONAL {
    ?creation cidoc-crm:P14_carried_out_by ?actor .
    ?creation lrmoo:R17_created ?expression .
    ?manifestation lrmoo:R4_embodies ?expression .
    ?manifestation cidoc-crm:P102_has_title ?title .
    ?title rdfs:label ?Title .
    OPTIONAL {
      ?creation cidoc-crm:P4_has_time-span ?time .
      ?time rdf:type cidoc-crm:E52_Time-Span .
      ?time ?property ?dateValue .
      FILTER (isLiteral(?dateValue))
      BIND(STR(?dateValue) AS ?Date)
    }
  }
}

"""

results = graph.query(sparql_query)

for result in results:
    Actor_id = result.get("actor")
    Actor = result.get("Actor")
    Roles = result.get("Role")
    Titles = result.get("Title")
    Dates = result.get("Date")

    print("Actor ID:", Actor_id)
    print("Actor:", Actor)
    print("Roles:", Roles)
    print("Titles:", Titles)
    print("Dates:", Dates)
    print("------")

Actor ID: https://isni.org/isni/0000000047415407
Actor: Gerrit van Rijn
Roles: Author
Titles: Atlas van Stolk : katalogus der historie-, spot- en zinneprenten betrekkelijk de geschiedenis van Nederland, verz. door A. van Stolk Cz.
Dates: 1895-1933
------
Actor ID: https://isni.org/isni/0000000051186344
Actor: Claudius Ptolemaeus
Roles: Author
Titles: Claudii Ptolemæi Alexandrini geographiae libri octo Græco-Latini
Dates: 1605
------
Actor ID: https://isni.org/isni/000000005731997X
Actor: Egbert Haubois
Roles: Cartographer
Titles: Afbeeldinge der stadt Groningen met de omliggende fortressen : 1652
Dates: 1652
------
Actor ID: https://isni.org/isni/0000000066999217
Actor: Cornelis Mortier
Roles: Publisher
Titles: None
Dates: None
------
Actor ID: https://isni.org/isni/0000000072511492
Actor: Albert Kampen
Roles: Mapmaker
Titles: Italia
Dates: None
------
Actor ID: https://isni.org/isni/0000000109366197
Actor: C. Koeman
Roles: Author
Titles: Atlantes Neerlandici
Dates: 1997-
------
Actor 

In [25]:
sparql_query = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX lrmoo: <http://iflastandards.info/ns/lrm/lrmoo/>
PREFIX cidoc-crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX crmdig: <http://www.cidoc-crm.org/extensions/crmdig/>

SELECT ?Digital_Image ?previous_version ?Creation_Date ?Modification_Date
WHERE {
  ?item lrmoo:R7_exemplifies ?edition .
  ?digitization crmdig:L1_digitized ?item .
  ?digitization crmdig:L20_has_created ?Digital_Image .
  ?Digital_Image cidoc-crm:P102_has_title ?title .
  ?title rdf:type cidoc-crm:E35_Title .
  ?title rdfs:label ?Title .
  ?Digital_Image cidoc-crm:P48_has_preferred_identifier ?Identifier .
  ?Digital_Image rdf:type lrmoo:F3_Manifestation .
  ?Digital_Image rdf:type crmdig:D9_Data_Object .

  OPTIONAL{
      ?digitization crmdig:L20_has_created ?Digital_Image .
      ?digitization cidoc-crm:P4_has_time-span ?current_timespan .
      ?current_timespan rdf:type cidoc-crm:E52_Time-Span .
      ?current_timespan cidoc-crm:P82_at_some_time_within ?Modification_Date .
        }
        
  OPTIONAL{
      ?Digital_Image lrmoo:R78_has_alternate ?previous_version .
      ?pre_digitization crmdig:L20_has_created ?previous_version .
      ?pre_digitization cidoc-crm:P4_has_time-span ?pre_timespan .
      ?pre_timespan rdf:type cidoc-crm:E52_Time-Span .
      ?pre_timespan cidoc-crm:P82_at_some_time_within ?Creation_Date .
        }
}
"""

results = graph.query(sparql_query)
for result in results:
    digital_image = result.get("Digital_Image")
    previous_version = result.get("previous_version")
    creation_date = result.get("Creation_Date")
    modification_date = result.get("Modification_Date")
    
    print("Digital Image:", digital_image)
    print("Previous Version:", previous_version)
    print("Creation Date:", creation_date)
    print("Modification Date:", modification_date)
    print("\n")

Digital Image: http://cdm21053.contentdm.oclc.org/cdm/ref/collection/Kaarten/id/1224
Previous Version: https://cdm21053.contentdm.oclc.org/digital/collection/Kaarten/id/1224#previous1224
Creation Date: 2012-10-24
Modification Date: 2012-10-24


Digital Image: http://cdm21053.contentdm.oclc.org/cdm/ref/collection/Kaarten/id/1492
Previous Version: https://cdm21053.contentdm.oclc.org/digital/collection/Kaarten/id/1492#previous1492
Creation Date: 2012-10-24
Modification Date: 2012-10-26


Digital Image: http://cdm21053.contentdm.oclc.org/cdm/ref/collection/Kaarten/id/1498
Previous Version: https://cdm21053.contentdm.oclc.org/digital/collection/Kaarten/id/1498#previous1498
Creation Date: 2012-10-24
Modification Date: 2012-10-26


Digital Image: http://cdm21053.contentdm.oclc.org/cdm/ref/collection/Kaarten/id/2605
Previous Version: https://cdm21053.contentdm.oclc.org/digital/collection/Kaarten/id/2605#previous2605
Creation Date: 2017-01-23
Modification Date: 2018-06-01


Digital Image: http:

In [26]:
sparql_query = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX lrmoo: <http://iflastandards.info/ns/lrm/lrmoo/>
PREFIX cidoc-crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX crmdig: <http://www.cidoc-crm.org/extensions/crmdig/>

SELECT ?Identifier ?Title 
       (GROUP_CONCAT(DISTINCT ?spatial_keyword; SEPARATOR=", ") AS ?Spatial_Keyword_IRIs)
       (GROUP_CONCAT(DISTINCT ?Spatial_Keyword; SEPARATOR=", ") AS ?Spatial_Keywords)
WHERE {
  ?map cidoc-crm:P138_represents ?spatial_keyword . 
  ?map cidoc-crm:P102_has_title ?title .
  ?title rdf:type cidoc-crm:E35_Title .
  ?title rdfs:label ?Title .
  ?map cidoc-crm:P48_has_preferred_identifier ?Identifier .
  ?spatial_keyword rdf:type cidoc-crm:E53_Place .
  ?spatial_keyword rdfs:label ?Spatial_Keyword .
      OPTIONAL {
        ?spatial_keyword rdfs:label ?label .
        FILTER(?label = "Netherlands")
      }
      OPTIONAL {
        ?spatial_keyword cidoc-crm:P10_falls_within ?parent_place .
        ?parent_place rdfs:label "Netherlands" .
      }
      FILTER(bound(?label) || bound(?parent_place))
      } 
      GROUP BY ?Identifier ?Title

"""

results = graph.query(sparql_query)

for result in results:
    Identifier = result.get("Identifier")
    Title = result.get("Title")
    Spatial_Keyword = result.get("Spatial_Keywords")
    Spatial_Keyword_IRIs = result.get("Spatial_Keyword_IRIs")
    print("Identifier:", Identifier)
    print("Title:", Title)
    print("Spatial Keywords:", Spatial_Keyword)
    print("Place ID:", Spatial_Keyword_IRIs)
    print("\n")

Identifier: http://cdm21053.contentdm.oclc.org/cdm/ref/collection/Kaarten/id/1224
Title: Afbeeldinge der stadt Groningen met de omliggende fortressen : 1652
Spatial Keywords: Nieuweschans, Netherlands, Groningen (city), Delfzijl, Coevorden, Bourtange
Place ID: https://www.geonames.org/2750250, https://www.geonames.org/2750405, https://www.geonames.org/2755251, https://www.geonames.org/2757339, https://www.geonames.org/2757936, https://www.geonames.org/2758514


Identifier: http://cdm21053.contentdm.oclc.org/cdm/ref/collection/Kaarten/id/2790
Title: Europae Tabula IIII
Spatial Keywords: Friesland
Place ID: https://www.geonames.org/2755812


Identifier: http://cdm21053.contentdm.oclc.org/cdm/ref/collection/Kaarten/id/1492
Title: Tweede stuk der beleegering van Bergen op Zoom : met de afbeeldinge der Franse attaque op de Linie en Roversfort
Spatial Keywords: Bergen op Zoom (city)
Place ID: https://www.geonames.org/2759144


Identifier: http://cdm21053.contentdm.oclc.org/cdm/ref/collection

In [47]:
sparql_query = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX lrmoo: <http://iflastandards.info/ns/lrm/lrmoo/>
PREFIX cidoc-crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX crmdig: <http://www.cidoc-crm.org/extensions/crmdig/>

SELECT ?Identifier ?Title ?Specific_Map_Type
WHERE {
  ?item cidoc-crm:P2_has_type ?map_type . 
  ?map_type cidoc-crm:P127_has_broader_term ?map .
  ?map rdfs:label ?label
  FILTER(?label = "Map")
  ?map_type rdfs:label ?Specific_Map_Type .
  ?item cidoc-crm:P48_has_preferred_identifier ?Identifier .
  ?item cidoc-crm:P102_has_title ?title .
  ?title rdfs:label ?Title .
      } 

"""

results = graph.query(sparql_query)

for result in results:
    Identifier = result.get("Identifier")
    Title = result.get("Title")
    Specific_Map_Type = result.get("Specific_Map_Type")
    print("Identifier:", Identifier)
    print("Title:", Title)
    print("Specific Map Type:", Specific_Map_Type)
    print("\n")

Identifier: http://cdm21053.contentdm.oclc.org/cdm/ref/collection/Kaarten/id/2605
Title: Italia
Specific Map Type: Wandkaart


Identifier: http://cdm21053.contentdm.oclc.org/cdm/ref/collection/Kaarten/id/2791
Title: Wereldkaart 1509
Specific Map Type: Wereldkaart




In [28]:
sparql_query = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX lrmoo: <http://iflastandards.info/ns/lrm/lrmoo/>
PREFIX cidoc-crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX crmdig: <http://www.cidoc-crm.org/extensions/crmdig/>

SELECT ?Identifier ?Title ?sheet ?number
WHERE {
  ?physical_item lrmoo:R7_exemplifies ?edition .
  ?digitization crmdig:L1_digitized ?physical_item .
  ?digitization crmdig:L20_has_created ?digital_Image .
  ?digital_Image cidoc-crm:P102_has_title ?title .
  ?title rdf:type cidoc-crm:E35_Title .
  ?title rdfs:label ?Title .
  ?digital_Image cidoc-crm:P48_has_preferred_identifier ?Identifier .
  
  OPTIONAL {
        ?physical_item cidoc-crm:P57_has_number_of_parts ?sheet .
        FILTER(?sheet > 1)
      }
  OPTIONAL {
        ?digital_Image cidoc-crm:P43_has_dimension ?dimension .
        ?dimension cidoc-crm:P2_has_type ?type .
        ?type rdfs:label "Number of maps" .
        ?dimension cidoc-crm:P90_has_value ?number .
        FILTER(?number > 1)
      }
      FILTER(bound(?sheet) || bound(?number))
      } 
"""

results = graph.query(sparql_query)

for result in results:
    Identifier = result.get("Identifier")
    Title = result.get("Title")
    sheet = result.get("sheet")
    number = result.get("number")
    print("Identifier:", Identifier)
    print("Title:", Title)
    print("Sheet:", sheet)
    print("Number of Maps:", number)

Identifier: http://cdm21053.contentdm.oclc.org/cdm/ref/collection/Kaarten/id/1492
Title: Tweede stuk der beleegering van Bergen op Zoom : met de afbeeldinge der Franse attaque op de Linie en Roversfort
Sheet: None
Number of Maps: 3
Identifier: http://cdm21053.contentdm.oclc.org/cdm/ref/collection/Kaarten/id/1498
Title: Grand theatre de la guerre sur les frontieres de France ÿ& d'Allemagne ou carte nouvelle du cours du Rhin et des païs circonvoisins contenant la Haute ÿ Basse Alsace, ...
Sheet: 2
Number of Maps: None


In [48]:
sparql_query = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX lrmoo: <http://iflastandards.info/ns/lrm/lrmoo/>
PREFIX cidoc-crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX crmdig: <http://www.cidoc-crm.org/extensions/crmdig/>

SELECT ?Title ?size
WHERE {
  ?item cidoc-crm:P43_has_dimension ?dimension . 
  ?dimension cidoc-crm:P2_has_type ?type .
  ?type rdfs:label "Map Size" .
  ?dimension cidoc-crm:P3_has_note ?size .
  ?item cidoc-crm:P102_has_title ?title .
  ?title rdfs:label ?Title
      } 

"""

results = graph.query(sparql_query)

for result in results:
    Title = result.get("Title")
    size = result.get("size")
    print("Title:", Title)
    print("Size:", size)
    print("\n")

Title: Afbeeldinge der stadt Groningen met de omliggende fortressen : 1652
Size: totaal 45,5 x 55 cm, plattegrond van de stad 25,5 x 35 cm


Title: Afbeeldinge der stadt Groningen met de omliggende fortressen : 1652
Size: totaal 45,5 x 55 cm, plattegrond van de stad 25,5 x 35 cm


Title: Tweede stuk der beleegering van Bergen op Zoom : met de afbeeldinge der Franse attaque op de Linie en Roversfort
Size: 20 x 24 cm


Title: Tweede stuk der beleegering van Bergen op Zoom : met de afbeeldinge der Franse attaque op de Linie en Roversfort
Size: 20 x 24 cm


Title: Grand theatre de la guerre sur les frontieres de France ÿ& d'Allemagne ou carte nouvelle du cours du Rhin et des païs circonvoisins contenant la Haute ÿ Basse Alsace, ...
Size: 137,5 x 124 cm


Title: Grand theatre de la guerre sur les frontieres de France ÿ& d'Allemagne ou carte nouvelle du cours du Rhin et des païs circonvoisins contenant la Haute ÿ Basse Alsace, ...
Size: 137,5 x 124 cm


Title: Italia
Size: 168 x 148 cm, stok